# Introduction

- **20210805 cholian**
目前以一级行业进行了筛选，未做二级行业，回头做一下二级行业试一下

- **20211121 cholian**
申万二级市场参数出现问题（系tushare数据库调用接口调整），修复bug

- **20211214 cholian**
申万二级市场index_member()数据为更新，系tushare数据库问题，目前已修复bug

# Package

In [3]:
# conda install tushare

In [1]:
import tushare as ts
import pandas as pd
# import matplotlib.pyplot as plt
# import datetime
import numpy as np
import time
import datetime

In [179]:
pro = ts.pro_api('')

## 参数选取

##### --------------------------------- <font color='red'>需要手动更新的地方 start </font>------------------------------------------------------

### 日期参数 （每周更新）

In [26]:
# 日期参数
# 提取上周五股票的原因是因为，周一要计算净流入的话，就要和上周五进行对比，因此，需要额外的把上周五的也提取出来
start_date ="20211203" # 上周五的日期
end_date = "20211210" # 最新周五的日期

### 行业股票参数（可选）

In [27]:
# 选取前几名行业 （默认为5）
top_industry = 5

# 选取前几名的股票
top_share = 5

##### ---------------------------------<font color='blue'>需要手动更新的地方 end </font> ------------------------------------------------------

## 准备函数

In [49]:
day = (pd.to_datetime(end_date)- pd.to_datetime(start_date)).days

In [157]:
def sw_indurstry( level = "L1",pro = None):
    #获取申万一级行业列表
    # -- Cholian
    # old: SW: the SW industry list (2014 version)
    # new: SW2021: the SW industry list (2021 version)
    df_first = pro.index_classify(level=level, src='SW2021')
    df_first_detail = pd.DataFrame()
    
    for i in df_first['index_code']:

        a = pro.index_member(index_code=i)
        print(i,'industry finished')
        # print('Sleep 2s')
        time.sleep(2)
        
        df_first_detail = pd.concat([a,df_first_detail])
        
    #SW2021 还未更新，存在个股多个行业的现象
    # -- Cholian
    # 解决办法：按照时间排序保留最后的行业代码
    df_first_detail = df_first_detail.drop_duplicates(subset='con_code', keep="last")
        
    return df_first_detail

def trade_datetime(end_date='20210714',period = 7):
    
    end_date = pd.to_datetime(end_date)
    dates = end_date - pd.to_timedelta(np.arange(period), unit = 'D')
    dates = dates.strftime('%Y%m%d')

    return dates


def pivot_data(data,col_pivot,dates_pivot,value):
    dates = pd.unique(data[dates_pivot])
    result = pd.DataFrame(columns=pd.unique(data[col_pivot]))
    result['date'] = ''
    
    for i in dates:

        daily = data[data[dates_pivot] == i]
        data_pivot = daily.groupby(col_pivot)[value].sum()
        
        a = pd.DataFrame(data_pivot).T
        a['date'] = i
        result = pd.concat([result,a])
    return result

def find_top(n,data):
    top_list = pd.DataFrame()
    for i,day in enumerate(data.index):
        top_data = data.iloc[i].sort_values(ascending = False).head(n).index
        top_list[day] = top_data
        
    return top_list

In [158]:
%%time
# 申万行业数据
df_first_detail = sw_indurstry( level = "L1",pro = pro)
df_second_detail = sw_indurstry( level = "L2",pro = pro)

801760.SI industry finished
801770.SI industry finished
801950.SI industry finished
801960.SI industry finished
801970.SI industry finished
801980.SI industry finished
801016.SI industry finished
801015.SI industry finished
801011.SI industry finished
801014.SI industry finished
801012.SI industry finished
801017.SI industry finished
801018.SI industry finished
801019.SI industry finished
801033.SI industry finished
801034.SI industry finished
801032.SI industry finished
801036.SI industry finished
801037.SI industry finished
801038.SI industry finished
801039.SI industry finished
801043.SI industry finished
801044.SI industry finished
801045.SI industry finished
801051.SI industry finished
801055.SI industry finished
801053.SI industry finished
801054.SI industry finished
801056.SI industry finished
801081.SI industry finished
801083.SI industry finished
801084.SI industry finished
801082.SI industry finished
801085.SI industry finished
801086.SI industry finished
801093.SI industry f

In [180]:
%%time
data_all = pd.DataFrame()
dates = trade_datetime(end_date=end_date,period = day+1)

drop_cols = ['code','con_code_x','in_date_x','out_date_x','con_code_y',
         'in_date_y','out_date_y','open','high','low','pre_close','pct_chg',
        'vol_y','amount','change']

for count, i in enumerate(dates):

    #北向数据
    df_sh = pro.hk_hold(trade_date = i,exchange = 'SH')
    df_sz = pro.hk_hold(trade_date = i,exchange = 'SZ')

    # 股票量价数据
    data = pd.concat([df_sh,df_sz])
    df_price = pro.daily(trade_date=i)

    data = data.merge(df_price, how='inner', on=['ts_code','trade_date'])
    

    # 数据合并
    data = data.merge(df_first_detail, how='inner', left_on ="ts_code",right_on='con_code')
    data = data.merge(df_second_detail, how='inner', left_on ="ts_code",right_on='con_code')

    # 数据清洗
    data = data.drop(drop_cols,axis=1)

    data_all = pd.concat([data_all,data])   
    
    # if count%2 == 0:
    #     print('time sleep')
    #     time.sleep(60)
    time.sleep(1)

    print(i, "finished")
print(f"{data_all['trade_date'].iloc[0]} 至 {data_all['trade_date'].iloc[-1]}的数据已经提取完毕")

data = data_all

20211210 finished
20211209 finished
20211208 finished
20211207 finished
20211206 finished
20211205 finished
20211204 finished
20211203 finished
20211210 至 20211203的数据已经提取完毕
CPU times: user 755 ms, sys: 107 ms, total: 861 ms
Wall time: 36.8 s


In [183]:
%%time
data.columns = ['date','code','name','north_holding','ratio','exchange','close','sw_first','sw_second']

shares = pd.unique(data['code'])

data_net_hd = pd.DataFrame()
for i in range(len(shares)):
    share_table = data[data['code'] == shares[i]]
    share_table = share_table.sort_values(by='date',ascending=True)
    share_table['net_hd'] = share_table['north_holding'].diff()
    data_net_hd = pd.concat([data_net_hd,share_table])


data_net_hd['net_cf'] = data_net_hd['net_hd']* data_net_hd['close']

# 去除NaN值
data = data_net_hd.dropna()

CPU times: user 4.78 s, sys: 38.4 ms, total: 4.82 s
Wall time: 4.83 s


In [185]:
%%time
# 创建一个全日期的空序列
share_pivot = pd.DataFrame(pd.unique(data['date']), columns={'date'})
shares = pd.unique(data['code'])

for share in shares:
    share_net_cf = data[data['code'] == share][['date','net_cf']].sort_values(by = 'date',ascending=True)

    share_net_cf = share_net_cf.rename(columns={'net_cf': share})

    share_pivot = share_pivot.merge(share_net_cf,how = 'outer',on = 'date')
    
# recording
    if share_pivot.shape[1] % 500 == 0:
        print(share_pivot.shape[1])

500
1000
1500
2000
CPU times: user 1min 58s, sys: 793 ms, total: 1min 59s
Wall time: 1min 59s


In [187]:
share_pivot = share_pivot.fillna(0)

# share_pivot = pivot_data(data,'code','date','net_cf')

share_pivot['date'] = pd.to_datetime(share_pivot['date'],format= "%Y%m%d")

share_pivot =  share_pivot.set_index('date',drop = True)
share_pivot_w = share_pivot.resample("W-FRI").sum()
sw_first = pivot_data(data,'sw_first','date','net_cf')

sw_first['date'] = pd.to_datetime(sw_first['date'],format='%Y%m%d')
sw_first = sw_first.set_index('date',drop=True)
sw_first_w = sw_first.resample("W-FRI").sum()

## 筛选行业

In [192]:
industry_code_w = find_top(top_industry,sw_first_w)

## 筛选股票

In [193]:
share_code = pd.DataFrame()

for each_day in range(industry_code_w.shape[1]):

    share_code_w = pd.DataFrame()

    for each_industry in industry_code_w.iloc[:,each_day]:

        #     获取分类的成份股
        industry_share = pd.unique(data[data['sw_first'] == each_industry]['code'])

        #     每个行业按照北向现金流净流入选出top n个股票
        share = share_pivot_w.loc[industry_code_w.columns[each_day],industry_share].sort_values(ascending = False).index[:top_share]

        share = pd.DataFrame(share,columns=[industry_code_w.columns[each_day]])

        
        share_code_w = pd.concat([share_code_w,share],axis = 0)
    share_code = pd.concat([share_code,share_code_w],axis = 1)
    if each_day % 20 == 0:
        print(each_day)

0


In [194]:
share_code = share_code.reset_index(drop=False)

In [195]:
result = pd.DataFrame()

# 记录rand和股票code
result['rank'] = share_code.iloc[:,0]
result['code'] = share_code.iloc[:,1]

share_name = data[['code','name']]

# 合并申万行业、股票名称
result = result.merge(data[['code','name']].drop_duplicates(),how='left',left_on='code',right_on='code')
result = result.merge(df_first_detail,how='left',left_on='code',right_on='con_code')
result = result.merge(df_second_detail,how='left',left_on='code',right_on='con_code')

share_name = data[['code','name']]

result = result.iloc[:,[0,1,2,3,7]]

result.columns = ['rank','code','name','sw_first_code','sw_second_code']

# 补充申万行业名称
sw_classify_first = pro.index_classify(level='L1', src='SW2021')
sw_classify_second = pro.index_classify(level='L2', src='SW2021')

result = result.merge(sw_classify_first[['index_code','industry_name']],how='left',left_on='sw_first_code',right_on='index_code')
result = result.merge(sw_classify_second[['index_code','industry_name']],how='left',left_on='sw_second_code',right_on='index_code')
result = result.drop(['sw_first_code','sw_second_code'],axis = 1)

result = result.rename(columns={'index_code_x': 'sw_first_code', 'industry_name_x': 'sw_first','index_code_y':'sw_second_code','industry_name_y':'sw_second'})

# 可供分析的表格

In [196]:
result['time'] = end_date

In [197]:
result.to_csv('北向资金流向股票.csv',encoding='utf_8_sig')
result

,rank,code,name,sw_first_code,sw_first,sw_second_code,sw_second,time
0,0,600519.SH,貴州茅台,801120.SI,食品饮料,801125.SI,白酒Ⅱ,20211210
1,1,000858.SZ,五 糧 液,801120.SI,食品饮料,801125.SI,白酒Ⅱ,20211210
2,2,002304.SZ,洋河股份,801120.SI,食品饮料,801125.SI,白酒Ⅱ,20211210
3,3,000799.SZ,酒鬼酒,801120.SI,食品饮料,801125.SI,白酒Ⅱ,20211210
4,4,600809.SH,山西汾酒,801120.SI,食品饮料,801125.SI,白酒Ⅱ,20211210
5,0,300059.SZ,東方財富,801790.SI,非银金融,801193.SI,证券Ⅱ,20211210
6,1,600030.SH,中信證券,801790.SI,非银金融,801193.SI,证券Ⅱ,20211210
7,2,601318.SH,中國平安,801790.SI,非银金融,801194.SI,保险Ⅱ,20211210
8,3,601688.SH,華泰證券,801790.SI,非银金融,801193.SI,证券Ⅱ,20211210
9,4,000776.SZ,廣發證券,801790.SI,非银金融,801193.SI,证券Ⅱ,20211210


# 保存以往的数据

In [198]:
data = pd.read_csv('历史股票池.csv',index_col=0,encoding='utf_8_sig')

In [199]:
data = pd.concat([data,result])
data.to_csv('历史股票池.csv',encoding='utf_8_sig')

# 筛选后的的股票

In [200]:
selected_share = [0,5,19, 20, 18, 22, 17, 24, 21, 12]
result.iloc[selected_share,:].to_csv('选出的股票.csv',encoding='utf_8_sig')
result.iloc[selected_share,:]

,rank,code,name,sw_first_code,sw_first,sw_second_code,sw_second,time
0,0,600519.SH,貴州茅台,801120.SI,食品饮料,801125.SI,白酒Ⅱ,20211210
5,0,300059.SZ,東方財富,801790.SI,非银金融,801193.SI,证券Ⅱ,20211210
19,4,300207.SZ,欣旺達,801730.SI,电力设备,801737.SI,电池,20211210
20,0,300124.SZ,匯川技術,801890.SI,机械设备,801078.SI,自动化设备,20211210
18,3,300450.SZ,先導智能,801730.SI,电力设备,801737.SI,电池,20211210
22,2,600031.SH,三一重工,801890.SI,机械设备,801077.SI,工程机械,20211210
17,2,002812.SZ,恩捷股份,801730.SI,电力设备,801737.SI,电池,20211210
24,4,603666.SH,億嘉和,801890.SI,机械设备,801078.SI,自动化设备,20211210
21,1,002008.SZ,大族激光,801890.SI,机械设备,801078.SI,自动化设备,20211210
12,2,000001.SZ,平安銀行,801780.SI,银行,801783.SI,股份制银行Ⅱ,20211210
